<a href="https://colab.research.google.com/github/Jiaye39/TimeSeriesAnalysis/blob/main/Bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bag of Words
We study 2 types of BoW vectors:
* **Raw Count**: actually count the number of occurences of each word in a text
* **TF-IDF**: adjust the raw count to favor words that appear a lot in a few documents, as opposed to those who appear a lot in all documents
* 关联package: NLTL sapcy tqdm eli5 gensim rich
* 介绍库基本用法+原理，实践回归+分析回归权重 (词性的neg或pos)

Basic import

In [ ]:
import numpy as np
import pandas as pd
import math
import requests

# Download Corpus

In [ ]:
r=requests.get('https://sherlock-holm.es/stories/plain-text/scan.txt')

assert r.status_code == 200

with open('scandal_in_bohemia.txt', 'w') as out:
    out.write(r.content.decode('utf-8'))
lines = [txt for txt in open('scandal_in_bohemia.txt') if len(txt.strip()) > 0]

print(lines[:20])

['                              A SCANDAL IN BOHEMIA\n', '                               Arthur Conan Doyle\n', '                                Table of contents\n', '                                     Chapter 1\n', '                                     Chapter 2\n', '                                     Chapter 3\n', '          CHAPTER I\n', '     To Sherlock Holmes she is always the woman. I have seldom heard him\n', '     mention her under any other name. In his eyes she eclipses and\n', '     predominates the whole of her sex. It was not that he felt any\n', '     emotion akin to love for Irene Adler. All emotions, and that one\n', '     particularly, were abhorrent to his cold, precise but admirably\n', '     balanced mind. He was, I take it, the most perfect reasoning and\n', '     observing machine that the world has seen, but as a lover he would\n', '     have placed himself in a false position. He never spoke of the softer\n', '     passions, save with a gibe and a sneer. T

In [ ]:
#Define First Paragraph
par=''.join([x.strip() for x in lines[7:25]])

# NLTK
*   punkt：这是一个预训练的分词器模型，用于将文本分割成句子和单词。这是许多NLP任务的基础步
*  punkt_tab：这是另一个与punkt相关的分词器，可能用于处理特定格式的文本或提供额外的分词功能




In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True



*   sent_tokenize: The sentence tokenizer takes care to split a text into sentences.
*   word_tokenize: The word tokenizer takes care to split a text into words.

*   拆解文本->向量化





In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
nltk_sentences=sent_tokenize(par)
nltk_words=word_tokenize(par)
print(nltk_sentences,'\n')
print(nltk_words)

['To Sherlock Holmes she is always the woman.', 'I have seldom heard himmention her under any other name.', 'In his eyes she eclipses andpredominates the whole of her sex.', 'It was not that he felt anyemotion akin to love for Irene Adler.', 'All emotions, and that oneparticularly, were abhorrent to his cold, precise but admirablybalanced mind.', 'He was, I take it, the most perfect reasoning andobserving machine that the world has seen, but as a lover he wouldhave placed himself in a false position.', 'He never spoke of the softerpassions, save with a gibe and a sneer.', "They were admirable thingsfor the observer--excellent for drawing the veil from men's motivesand actions.", 'But for the trained reasoner to admit such intrusionsinto his own delicate and finely adjusted temperament was tointroduce a distracting factor which might throw a doubt upon all hismental results.', 'Grit in a sensitive instrument, or a crack in one ofhis own high-power lenses, would not be more disturbing th

# Spacy


*   en_core_web_sm 包含处理en文本的所有组件与数据
*   zh_core_web_sm 处理中文zh版本



In [ ]:
import spacy
nlp=spacy.load('en_core_web_sm')
doc=nlp(par)



*   spacy_sentences (doc.sents) : 文本拆解为句子 -> 向量化
*   spacy_tokens (x for x in xxxx[i]) : 句子拆解为tokens -> 向量化



In [ ]:
spacy_sentences=list(doc.sents)
spacy_tokens=[x for x in spacy_sentences[0]]
print(spacy_sentences,'\n')
print(spacy_tokens)

[To Sherlock Holmes she is always the woman., I have seldom heard himmention her under any other name., In his eyes she eclipses andpredominates the whole of her sex., It was not that he felt anyemotion akin to love for Irene Adler., All emotions, and that oneparticularly, were abhorrent to his cold, precise but admirablybalanced mind., He was, I take it, the most perfect reasoning andobserving machine that the world has seen, but as a lover he wouldhave placed himself in a false position., He never spoke of the softerpassions, save with a gibe and a sneer., They were admirable thingsfor the observer--excellent for drawing the veil from men's motivesand actions., But for the trained reasoner to admit such intrusionsinto his own delicate and finely adjusted temperament was tointroduce a distracting factor which might throw a doubt upon all hismental results., Grit in a sensitive instrument, or a crack in one ofhis own high-power lenses, would not be more disturbing than a strongemotion 

# Sklearn Generalities （CountVectorizer&TfidfVectorizer）
Classes likes `CountVectorizer` or `TfidfVectorizer` works in the following way:
* Instantiate an object with specific parameters (`v = CountVectorizer(...)`)
* Fit this object to your corpus = learn the vocabulary (method `v.fit(...)`)
* Transform any piece of text you have into a vector (method `v.transform()`)
*   **用CountVectorizer或者TfidfVectorizer做特征提取，文本转化为bow后才可以进行回归等操作**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

def 2 function for below analysis

In [ ]:
def show_vocabulary(vectorizer):
    words = vectorizer.get_feature_names_out()

    print(f'Vocabulary size: {len(words)} words')

    # we can print ~10 words per line
    for l in np.array_split(words, math.ceil(len(words) / 10)):
        print(''.join([f'{x:<15}' for x in l]))

In [ ]:
import os
os.environ["FORCE_COLOR"] = "1"

from termcolor import colored

def show_bow(vectorizer, bow):
    words = vectorizer.get_feature_names_out()

    # we can print ~8 words + coefs per line
    for l in np.array_split(list(zip(words, bow)), math.ceil(len(words) / 8)):
        print(' | '.join([colored(f'{w:<15}:{n:>2}', 'grey') if int(n) == 0 else colored(f'{w:<15}:{n:>2}', on_color='on_yellow', attrs=['bold']) for w, n in l ]))

def show_bow_float(vectorizer, bow):
    words = vectorizer.get_feature_names_out()

    # we can print ~6 words + coefs per line
    for l in np.array_split(list(zip(words, bow)), math.ceil(len(words) / 6)):
        print(' | '.join([colored(f'{w:<15}:{float(n):>0.2f}', 'grey') if float(n) == 0 else colored(f'{w:<15}:{float(n):>0.2f}', on_color='on_yellow', attrs=['bold']) for w, n in l ]))


# Raw Count
* We take a text, any text, and represent it as a vector
* Each text is represented by a vector with **N** dimensions
* Each dimension is representative of **1 word** of the vocabulary
* The coefficient in dimension **k** is the number of times the word at index **k** in the vocabulary is seen in the represented text

Eg: Reduced Vocabulary. (corpus: 1st paragraph of book, document:1 sentence)

In [ ]:
count_small= CountVectorizer(lowercase=True)
count_small.fit(nltk_sentences)
show_vocabulary(count_small)

Vocabulary size: 126 words
abhorrent      actions        adjusted       adler          admirable      admirablybalancedadmit          akin           all            always         
and            andobserving   andpredominatesandquestionableany            anyemotion     as             be             but            cold           
crack          delicate       distracting    disturbing     doubt          drawing        dubious        eclipses       emotions       excellent      
eyes           factor         false          felt           finely         for            from           gibe           grit           has            
have           he             heard          her            high           himmention     himself        his            hismental      holmes         
in             instrument     intrusionsinto irene          is             it             late           lenses         love           lover          
machine        memory         men            might          mind 

The option `lowercase` sets up one behavior of the raw count: do we consider `And` to be different than `and`?

* `lowercase=False` gives 134 unique words in the vocabulary
* `lowercase=True` gives 127 unique words

In [ ]:
s = nltk_sentences[0]

print(f'Text: "{s}"')
bow = count_small.transform([s])
print(f'BoW Shape: {bow.shape}')
bow = bow.toarray()   # From sparse matrix to dense matrix (Careful with MEMORY)
print(f'BoW Vector: {bow}')

Text: "To Sherlock Holmes she is always the woman."
BoW Shape: (1, 126)
BoW Vector: [[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]]


In [ ]:
show_bow(count_small,bow[0])

abhorrent      : 0 | actions        : 0 | adjusted       : 0 | adler          : 0 | admirable      : 0 | admirablybalanced: 0 | admit          : 0 | akin           : 0
all            : 0 | always         : 1 | and            : 0 | andobserving   : 0 | andpredominates: 0 | andquestionable: 0 | any            : 0 | anyemotion     : 0
as             : 0 | be             : 0 | but            : 0 | cold           : 0 | crack          : 0 | delicate       : 0 | distracting    : 0 | disturbing     : 0
doubt          : 0 | drawing        : 0 | dubious        : 0 | eclipses       : 0 | emotions       : 0 | excellent      : 0 | eyes           : 0 | factor         : 0
false          : 0 | felt           : 0 | finely         : 0 | for            : 0 | from           : 0 | gibe           : 0 | grit           : 0 | has            : 0
have           : 0 | he             : 0 | heard          : 0 | her            : 0 | high           : 0 | himmention     : 0 | himself        : 0 | his            : 0
hi

# TF-IDF
The basic for TF-IDF is that cosine similarity with raw count coefficients puts too much emphasis on the number of occurences of a word within a document.

Repeating a word will artifically increase the cosine similarity with any text containing this word.

Consider which word would be important:
1. One that is repeated a lot and equally present in each document
1. One that appears a lot only in a few document
TF-IDF computes coefficients:
* Low values for common words (ie present in the document, but quite common over the corpus)
* High values for uncommon words (ie present in the document, but not common over the corpus)

We consider one specific document, and one specific word.

* **TF = Term Frequency**: the number of times the word appears in the document
* **DF = Document Frequency**: the number of document in the corpus, in which the word appears
* **IDF = Inverse Document Frequency**: the inverse of the Document Frequency.

Logarithms are introduced, to reflect that 100 times a word does not deliver 100 times the information.

Given a word **w**, a document **d** in a corpus of **D** documents:

$\textrm{TF-IDF(w, d) = TF(w, d) * IDF(w)}$

$
\begin{align}
\textrm{IDF(w) = log} \left( \frac{1 + \textrm{D}}{1 + \textrm{DF(w)}} \right) + 1
\end{align}
$

In [ ]:
tfidf=TfidfVectorizer()
tfidf.fit(nltk_sentences)
show_vocabulary(tfidf)

Vocabulary size: 126 words
abhorrent      actions        adjusted       adler          admirable      admirablybalancedadmit          akin           all            always         
and            andobserving   andpredominatesandquestionableany            anyemotion     as             be             but            cold           
crack          delicate       distracting    disturbing     doubt          drawing        dubious        eclipses       emotions       excellent      
eyes           factor         false          felt           finely         for            from           gibe           grit           has            
have           he             heard          her            high           himmention     himself        his            hismental      holmes         
in             instrument     intrusionsinto irene          is             it             late           lenses         love           lover          
machine        memory         men            might          mind 

In [ ]:
s = nltk_sentences[0]

print(f'Text: "{s}"')
bow = tfidf.transform([s])
print(f'BoW Shape: {bow.shape}')
bow = bow.toarray()   # From sparse matrix to dense matrix (Careful with MEMORY)
print(f'BoW Vector: {bow}')

Text: "To Sherlock Holmes she is always the woman."
BoW Shape: (1, 126)
BoW Vector: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.40271589 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.40271589 0.         0.         0.         0.
  0.40271589 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0. 

In [ ]:
show_bow_float(tfidf,bow[0])

abhorrent      :0.00 | actions        :0.00 | adjusted       :0.00 | adler          :0.00 | admirable      :0.00 | admirablybalanced:0.00
admit          :0.00 | akin           :0.00 | all            :0.00 | always         :0.40 | and            :0.00 | andobserving   :0.00
andpredominates:0.00 | andquestionable:0.00 | any            :0.00 | anyemotion     :0.00 | as             :0.00 | be             :0.00
but            :0.00 | cold           :0.00 | crack          :0.00 | delicate       :0.00 | distracting    :0.00 | disturbing     :0.00
doubt          :0.00 | drawing        :0.00 | dubious        :0.00 | eclipses       :0.00 | emotions       :0.00 | excellent      :0.00
eyes           :0.00 | factor         :0.00 | false          :0.00 | felt           :0.00 | finely         :0.00 | for            :0.00
from           :0.00 | gibe           :0.00 | grit           :0.00 | has            :0.00 | have           :0.00 | he             :0.00
heard          :0.00 | her            :0.00 | 

Display the IDF of some words.

* High IDF = word that appears in few documents
* Low IDF = word that appears in most of documents

In [ ]:
words = tfidf.get_feature_names_out()
word = input('Word: ').lower()

if word in words:
    k = list(words).index(word)
    print(f'IDF({words[k]}) = {tfidf.idf_[k]}')
else:
    print('Not in vocabulary')

Word: my
Not in vocabulary


More than one TF_IDF:

There is a family of TF-IDF formulas.

Another example is the **sublinear TF**, which is then:

$
\begin{align}
\textrm{TF(w, d) = 1 + log} \left( raw count \right)
\end{align}
$

In [ ]:
tfidf_sublinear = TfidfVectorizer(sublinear_tf=True)
tfidf_sublinear.fit(nltk_sentences)

TfidfVectorizer(sublinear_tf=True)

In [ ]:
s = nltk_sentences[0]

print(f'Text: "{s}"')
bow_sl = tfidf_sublinear.transform([s])
print(f'BoW Shape: {bow_sl.shape}')
bow_sl = bow_sl.toarray()   # From sparse matrix to dense matrix (Careful with MEMORY)
print(f'BoW Vector: {bow_sl}')

Text: "To Sherlock Holmes she is always the woman."
BoW Shape: (1, 126)
BoW Vector: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.40271589 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.40271589 0.         0.         0.         0.
  0.40271589 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0. 

In [ ]:
show_bow_float(tfidf_sublinear, bow_sl[0])

abhorrent      :0.00 | actions        :0.00 | adjusted       :0.00 | adler          :0.00 | admirable      :0.00 | admirablybalanced:0.00
admit          :0.00 | akin           :0.00 | all            :0.00 | always         :0.40 | and            :0.00 | andobserving   :0.00
andpredominates:0.00 | andquestionable:0.00 | any            :0.00 | anyemotion     :0.00 | as             :0.00 | be             :0.00
but            :0.00 | cold           :0.00 | crack          :0.00 | delicate       :0.00 | distracting    :0.00 | disturbing     :0.00
doubt          :0.00 | drawing        :0.00 | dubious        :0.00 | eclipses       :0.00 | emotions       :0.00 | excellent      :0.00
eyes           :0.00 | factor         :0.00 | false          :0.00 | felt           :0.00 | finely         :0.00 | for            :0.00
from           :0.00 | gibe           :0.00 | grit           :0.00 | has            :0.00 | have           :0.00 | he             :0.00
heard          :0.00 | her            :0.00 | 

In [ ]:
word = 'yet'

index = words.tolist().index(word)

bow = tfidf.transform([s]).toarray()

print(f'Word: "{word}"')
print(f'TF-IDF with Natural TF   = {bow[0][index]:0.4f}')
print(f'TF-IDF with Sublinear TF = {bow_sl[0][index]:0.4f}')

Word: "yet"
TF-IDF with Natural TF   = 0.0000
TF-IDF with Sublinear TF = 0.0000


In [ ]:
word = 'yet'
s = nltk_sentences[0]
s = s + ' '.join(100 * [word])

bow = tfidf.transform([s]).toarray()
bow_sl = tfidf_sublinear.transform([s]).toarray()

index = words.tolist().index(word)
print(f'Word: "{word}"')
print(f'TF-IDF with Natural TF   = {bow[0][index]:0.4f}')
print(f'TF-IDF with Sublinear TF = {bow_sl[0][index]:0.4f}')

Word: "yet"
TF-IDF with Natural TF   = 0.9997
TF-IDF with Sublinear TF = 0.9143


# World Count & Freguency

# tqdm

*   from tqdm.notebook import tqdm
*   它用于在循环中显示进度条，方便用户了解代码执行的进度，尤其是在处理大量数据时。
*   基本用法：括号包裹一个可迭代对象： for i in tqdm(range(10)):



In [ ]:
import requests
r = requests.get("http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz")
imdb_tgz = r.content
import io
import re
import tarfile

from tqdm.notebook import tqdm

good_file = re.compile(r"^aclImdb/(test|train)/(pos|neg)/.*\.txt$")

with tarfile.open(fileobj=io.BytesIO(r.content), mode="r:gz") as tgz:
    all_members = tgz.getmembers()
    data_files = list(filter(lambda x: x.isfile() and good_file.match(x.name) is not None, all_members))
    for f in tqdm(data_files):
        tgz.extract(f)

  0%|          | 0/50000 [00:00<?, ?it/s]

/tmp/ipython-input-3986837542.py:16: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tgz.extract(f)


In [ ]:
from sklearn.datasets import load_files
train_data, test_data = load_files("./aclImdb/train", encoding="utf-8"), load_files("./aclImdb/test", encoding="utf-8")

label2txt = {label: txt for label, txt in enumerate(train_data.target_names)}
txt2label = {txt: label for label, txt in label2txt.items()}
type(train_data)

X_train, y_train = train_data.data, train_data.target
X_test, y_test = test_data.data, test_data.target

The Data

In [ ]:
import numpy as np

print("TRAIN data:")
print("class balance: ", np.bincount(y_train))
print()
print("TEST data:")
print("class balance: ", np.bincount(y_train))

TRAIN data:
class balance:  [12500 12500]

TEST data:
class balance:  [12500 12500]


# eli5 & gensim
**后期 notebook will add more info about these 2 package**
* eli5: 这主要用于帮助调试和解释机器学习模型。它可以揭示模型内部的工作原理，例如哪些特征对模型的预测贡献最大，这对于理解和信任模型非常有用。

* gensim: 这是一个专注于主题建模和自然语言处理（NLP）的Python库。它能够处理大型文本语料库，并提供了一系列算法，例如Latent Semantic Analysis (LSA)、Latent Dirichlet Allocation (LDA) 和 Word2Vec 等，用于发现文本数据中的潜在结构、相似性以及词语之间的关系。

In [ ]:
!pip install eli5
!pip install gensim
from gensim.corpora import Dictionary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 49.4 MB/s eta 0:00:00


**How many word in our vocabulary?**

In [ ]:
X_train_tokenized=[word_tokenize(x) for x in tqdm(X_train)]

  0%|          | 0/25000 [00:00<?, ?it/s]

In [ ]:
d=Dictionary(X_train_tokenized)

# rich
做表example：

     from rich.console import Console

     from rich.table import Table

     table.add_column(' ', juetify=' ',style=' ')

justify : 对齐方式-----'left'(默认) 'right' 'center': 内容对齐位置

In [ ]:
import rich

from rich.console import Console
from rich.table import Table

N = 20

table = Table(title=f"Top-{N} Most Frequent Tokens ({d.num_docs} documents, {d.num_pos} tokens, {len(d)} words in dictionary)")

table.add_column("Token", justify="left", style="black")
table.add_column("Corpus Frequency", justify="right")
table.add_column("% of Tokens", justify="right")
table.add_column("Document Frequency", justify="right")
table.add_column("% of Documents", justify="right")

for token, frequency in d.most_common(n=N):
    percent_tokens = frequency / d.num_pos
    doc_frequency = d.dfs[d.token2id[token]]
    percent_doc = doc_frequency / d.num_docs
    table.add_row(token, str(frequency), f"{percent_tokens:.1%}", str(doc_frequency), f"{percent_doc:.1%}")

console = Console()
console.print(table)

 Top-20 Most Frequent Tokens (25000 documents, 7065344 tokens, 132088 words in  
                                  dictionary)                                   
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Token ┃ Corpus Frequency ┃ % of Tokens ┃ Document Frequency ┃ % of Documents ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ the   │           289359 │        4.1% │              24674 │          98.7% │
│ ,     │           275887 │        3.9% │              24054 │          96.2% │
│ .     │           236573 │        3.3% │              24685 │          98.7% │
│ and   │           156416 │        2.2% │              24041 │          96.2% │
│ a     │           156158 │        2.2% │              24060 │          96.2% │
│ of    │           143692 │        2.0% │              23685 │          94.7% │
│ to    │           133446 │        1.9% │              23423 │          93.7% │
│ is    │           109177 │        1.5% │              22469 │          89.9% │
│ /     │           102115 │        1.4% │              14711 │          58.8% │
│ >     │           102036 │        1.4% │              14681 │          58.7% │
│ <     │           101971 │        1.4% │              14671 │          58.7% │
│ br    │           101870 │        1.4% │              14665 │          58.7% │
│ in    │            87145 │        1.2% │              21744 │          87.0% │
│ I     │            81531 │        1.2% │              19245 │          77.0% │
│ it    │            76616 │        1.1% │              21022 │          84.1% │
│ that  │            70130 │        1.0% │              20051 │          80.2% │
│ 's    │            61985 │        0.9% │              17879 │          71.5% │
│ this  │            60544 │        0.9% │              20994 │          84.0% │
│ was   │            49994 │        0.7% │              16321 │          65.3% │
│ The   │            43999 │        0.6% │              17514 │          70.1% │
└───────┴──────────────────┴─────────────┴────────────────────┴────────────────┘

# Classify with Logistic Regression
**X_train 转化为 bow 才可回归**

In [ ]:
vec=CountVectorizer(lowercase=False,token_pattern=None,analyzer=word_tokenize)
X_train_bow=vec.fit_transform(X_train)

X_train_bow

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 3842836 stored elements and shape (25000, 132088)>

生成X_train_bow  稀疏矩阵第一行（即第一个训练文档）中，所有非零元素的列索引，列索引代表了在第一个文档中出现过的词汇表中的词语的 ID。

In [ ]:
X_train_bow[0].indices

array([ 57272,  18579,  93268, 131583, 124015, 123244,   3678,  78681,
       109717, 130246, 125648,  64768,  96521, 131061,  75376, 129944,
       123171,  74196,   3702,  69436,  99119, 120872, 128281, 116897,
       123016,  68287,   4116,  29521,  66374,  99053,  62622,  63291,
        57540,  63614,  97760, 101497,  62972,  82187,  88487, 130166,
        86484,  72924, 130601,  82932,  69457,  68174,  89479, 102325,
        73176, 103323, 130942,  71104,  73866,   7092,  64773,   4312,
         7120,  91279, 100753, 104513,  98446,  65742,  84129,  97931,
       122981,  81190,  59713,  58209,  85685,  91343, 110889, 107628,
        29006, 122784,  79447,  98326,  58166, 131602, 120881,  63235,
       122930,    562,      7,  81050, 122951,  83923, 126358, 102518,
         3110, 116715,  80272,  59307,  60267,  60658,  93923, 114296,
        23244,  87870, 130596, 122794,  87879], dtype=int32)

In [ ]:
console = Console(record=True, width=40)
table = Table(title=f"Tokens in sample")

table.add_column("Token ID", justify="right")
table.add_column("Token", justify="left")
table.add_column("Count", justify="right")

words = vec.get_feature_names_out()
for token_id, _ in zip(sorted(X_train_bow[0].indices), range(20)):
    table.add_row(str(token_id), words[token_id], str(X_train_bow[0][0, token_id]))

console.print(table)
console.save_svg("bow.svg", title="Bag of Words")

          Tokens in sample           
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Token ID ┃ Token          ┃ Count ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━┩
│        7 │ '              │     2 │
│      562 │ 'Elephant      │     1 │
│     3110 │ 'rationalistic │     1 │
│     3678 │ ,              │     2 │
│     3702 │ -              │     3 │
│     4116 │ .              │     4 │
│     4312 │ /              │     4 │
│     7092 │ <              │     4 │
│     7120 │ >              │     4 │
│    18579 │ Day            │     1 │
│    23244 │ Flawed         │     1 │
│    29006 │ In             │     1 │
│    29521 │ It             │     2 │
│    57272 │ Zero           │     1 │
│    57540 │ a              │     6 │
│    58166 │ actions        │     1 │
│    58209 │ actors         │     1 │
│    59307 │ almost         │     1 │
│    59713 │ and            │     2 │
│    60267 │ anything       │     1 │
└──────────┴────────────────┴───────┘

**加入TF-IDF 进table above**

用TfidfVectorizer

In [ ]:
vec_tfidf = TfidfVectorizer(
    lowercase=False,
    token_pattern=None,
    analyzer=word_tokenize
)

X_train_tfidf = vec_tfidf.fit_transform(X_train)
X_train_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3842836 stored elements and shape (25000, 132088)>

In [ ]:
console = Console(record=True, width=80)
table = Table(title=f"Tokens in sample")

table.add_column("Token ID", justify="right")
table.add_column("Token", justify="left")
table.add_column("Count (TF)", justify="right")
table.add_column("Doc Frequency (% corpus)", justify="right")
table.add_column("TF-IDF", justify="right")

words = vec.get_feature_names_out()
for token_id, _ in zip(sorted(X_train_bow[0].indices), range(20)):
    table.add_row(str(token_id), words[token_id], f"{X_train_bow[0][0, token_id]}", f"{d.dfs[d.token2id[words[token_id]]] / d.num_docs:.3%}", f"{X_train_tfidf[0][0, token_id]:.3f}")

console.print(table)
console.save_svg("tfidf.svg", title="Bag of Words - TF-IDF")

                               Tokens in sample                               
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Token ID ┃ Token          ┃ Count (TF) ┃ Doc Frequency (% corpus) ┃ TF-IDF ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│        7 │ '              │          2 │                  18.588% │  0.093 │
│      562 │ 'Elephant      │          1 │                   0.004% │  0.181 │
│     3110 │ 'rationalistic │          1 │                   0.004% │  0.181 │
│     3678 │ ,              │          2 │                  96.216% │  0.036 │
│     3702 │ -              │          3 │                  16.028% │  0.148 │
│     4116 │ .              │          4 │                  98.740% │  0.070 │
│     4312 │ /              │          4 │                  58.844% │  0.106 │
│     7092 │ <              │          4 │                  58.684% │  0.107 │
│     7120 │ >              │          4 │                  58.724% │  0.107 │
│    18579 │ Day            │          1 │                   0.880% │  0.100 │
│    23244 │ Flawed         │          1 │                   0.012% │  0.169 │
│    29006 │ In             │          1 │                  18.272% │  0.047 │
│    29521 │ It             │          2 │                  44.860% │  0.063 │
│    57272 │ Zero           │          1 │                   0.320% │  0.117 │
│    57540 │ a              │          6 │                  96.240% │  0.108 │
│    58166 │ actions        │          1 │                   1.080% │  0.096 │
│    58209 │ actors         │          1 │                  13.884% │  0.052 │
│    59307 │ almost         │          1 │                  10.140% │  0.057 │
│    59713 │ and            │          2 │                  96.164% │  0.036 │
│    60267 │ anything       │          1 │                   9.856% │  0.058 │
└──────────┴────────────────┴────────────┴──────────────────────────┴────────┘

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
reg=LogisticRegression(max_iter=1000)
reg.fit(X_train_bow,y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_test_bow=vec.transform(X_test)

In [ ]:
print(classification_report(y_true=y_test,y_pred=reg.predict(X_test_bow)))

              precision    recall  f1-score   support

           0       0.87      0.88      0.88     12500
           1       0.88      0.87      0.87     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



# Which words indicate a positive review ?
使用-eli5-解释回归reg的权重

* Let's use the coefficients of logistic regression
* $x_i$ are token counts, we know $x_i>0$
* $\alpha_i > 0$ = the presence of the $i$-th word of dictionary indicates a positive review (because it increases the probability that $x$ belongs to the positive class)
* $\alpha_i < 0$ = the presence of the $i$-th word of dictionary indicates a negative review (because it increases the probability that $x$ belongs to the negative class)
* $\alpha_i>0, \alpha_j > 0$ and $\alpha_i > \alpha_j$ = the $i$-th word of dictionary is **more** associated to a positive review than the $j$-th word


In [ ]:
import eli5
vec.get_feature_names = vec.get_feature_names_out
eli5.show_weights(reg,vec=vec,top=10,target_names=['negative','positive'])

In [ ]:
eli5.explain_prediction(reg, X_test[0], vec=vec, target_names=['negative', 'positive'])

Explanation(estimator='LogisticRegression(max_iter=1000)', description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='positive', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='fun', weight=np.float64(0.6504092153251091), std=None, value=np.float64(1.0)), FeatureWeight(feature='well', weight=np.float64(0.5626663354710332), std=None, value=np.float64(1.0)), FeatureWeight(feature='Patricia', weight=np.float64(0.39402358103437773), std=None, value=np.float64(1.0)), FeatureWeight(feature='beautiful', weight=np.float64(0.38284450326958086), std=None, value=np.float64(1.0)), FeatureWeight(feature='flick', weight=np.float64(0.3123142457893798), std=None, value=np.float64(1.0)), FeatureWeight(feature='together', weight=np.float64(0.29805122771867565), std=None, value=np.float64(1.0)), FeatureWeight(feature='hate', weight=np.float64(0.2748251242920735), std=None, value=np.float64(2.0)), FeatureWeight(feature='and', weight=np.float64(0.2562402743167084), std=None, value=np.float64(3.0)), FeatureWeight(feature='gorgeous', weight=np.float64(0.2402811149037552), std=None, value=np.float64(1.0)), FeatureWeight(feature='Like', weight=np.float64(0.2077830952098098), std=None, value=np.float64(1.0)), FeatureWeight(feature='hip', weight=np.float64(0.19560419595564946), std=None, value=np.float64(1.0)), FeatureWeight(feature='Miramax', weight=np.float64(0.1707640029250947), std=None, value=np.float64(1.0)), FeatureWeight(feature='Lisa', weight=np.float64(0.16826682264673326), std=None, value=np.float64(1.0)), FeatureWeight(feature='Congratulations', weight=np.float64(0.14033986998094036), std=None, value=np.float64(1.0)), FeatureWeight(feature='&', weight=np.float64(0.13306504750975723), std=None, value=np.float64(1.0)), FeatureWeight(feature='the', weight=np.float64(0.12092163082288612), std=None, value=np.float64(3.0)), FeatureWeight(feature='watch', weight=np.float64(0.11695382689785636), std=None, value=np.float64(1.0)), FeatureWeight(feature='she', weight=np.float64(0.09350650400809747), std=None, value=np.float64(2.0)), FeatureWeight(feature='up', weight=np.float64(0.08237224235283039), std=None, value=np.float64(1.0)), FeatureWeight(feature='as', weight=np.float64(0.06932072032306662), std=None, value=np.float64(2.0)), FeatureWeight(feature='!', weight=np.float64(0.060498220291700024), std=None, value=np.float64(1.0)), FeatureWeight(feature='Krueger', weight=np.float64(0.05624836650646787), std=None, value=np.float64(1.0)), FeatureWeight(feature='funky', weight=np.float64(0.05440092186490784), std=None, value=np.float64(1.0)), FeatureWeight(feature='is', weight=np.float64(0.04178166385778115), std=None, value=np.float64(1.0)), FeatureWeight(feature='-', weight=np.float64(0.03671555138809488), std=None, value=np.float64(1.0)), FeatureWeight(feature='work', weight=np.float64(0.03445024024574575), std=None, value=np.float64(1.0)), FeatureWeight(feature='<BIAS>', weight=np.float64(0.025546293663190788), std=None, value=np.float64(1.0)), FeatureWeight(feature='Goran', weight=np.float64(0.024323786580729662), std=None, value=np.float64(1.0)), FeatureWeight(feature=',', weight=np.float64(0.01963463631121367), std=None, value=np.float64(2.0)), FeatureWeight(feature='movie', weight=np.float64(0.01807556005989947), std=None, value=np.float64(1.0)), FeatureWeight(feature='her', weight=np.float64(0.01777925131333304), std=None, value=np.float64(1.0)), FeatureWeight(feature='Visnjic', weight=np.float64(0.0017810863065476374), std=None, value=np.float64(1.0)), FeatureWeight(feature='TALENTED', weight=np.float64(0.00017677157632016037), std=None, value=np.float64(1.0))], neg=[FeatureWeight(feature='clothing', weight=np.float64(-0.5810865624141083), std=None, value=np.float64(1.0)), FeatureWeight(feature='Heather', weight=np.float64(-0.5105162775046949), std=None, value=np.float64(2.0)), FeatureWeight(feature='Director', weight=np.float64(-0.3215998135608746), std=None, value=np.float64(1.0)), 